In [ ]:
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import tensorflow as tf
sess = tf.Session()

In [ ]:
from keras import backend as K
K.set_session(sess)

In [ ]:
from keras.models import Model
from keras.layers import Dense, Dropout, Input
from keras.layers import Conv2D, Flatten, MaxPooling2D

In [ ]:
from keras.datasets import mnist
from keras.utils import to_categorical

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
def reluConv2d(x, filters=16):
    return Conv2D(filters=filters, kernel_size=(3,3), 
                  activation='relu', padding='same')(x)

def mp2d(x):
    return MaxPooling2D(pool_size=(2,2), 
                        padding='same')(x)

def convMP(x, filters=16):
    return mp2d(reluConv2d(x, filters))

In [ ]:
input_img = Input(shape=(28,28,1))
x = convMP(input_img)
x = convMP(x)
x = convMP(x)
x = Flatten()(x)
final_cnn = Dense(10, activation='softmax')(x)

digits_cnnclf = Model(input_img, final_cnn, name='cnnClassifier')

In [ ]:
digits_cnnclf.compile(optimizer='adadelta',
                      loss='categorical_crossentropy',
                      metrics=['mse', 'accuracy'])

## load data

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))  # adapt this if using `channels_first` image data format
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))  # adapt this if using `channels_first` image data format

In [ ]:
y_train_1h = to_categorical(y_train)
y_test_1h = to_categorical(y_test)

In [ ]:
print(x_train.shape)
print(x_test.shape)

## Exercises

<div class='alert alert-block alert-info'>
1. `fit` the model to the **training data** using a `validation_split` of `.2` and 10 epochs. Remember to `shuffle` the data!  
2. Evaluate the model **on the test data**. Report the accuracy.  
3. Display a classification report for the **test** data.  
4. What is precision? recall? support?  
5. Plot a confusion matrix, using the *inferno* colormap.  
6. What is the total number of *trainable parameters* the model has? How many in each layer? Are there more or fewer parameters than in the previous model?
7. Save the your CNN model. In a new notebook, load your *simple model* and your *CNN model*. On which entries of the testing data do the two models differ in their predicitions? Visualize these images using `matplotlib`. 
</div>

In [ ]:
digits_cnnclf.fit(x_train, y_train_1h, epochs=10, batch_size=256, 
               shuffle=True, validation_split=.2)

In [ ]:
score=digits_cnnclf.evaluate(x_test, y_test_1h)

In [ ]:
import pandas as pd
pd.DataFrame(data=[score], 
             columns=digits_cnnclf.metrics_names)

In [ ]:
y_proba = digits_cnnclf.predict(x_test)
y_pred = np.argmax(y_proba, axis=-1)

In [ ]:
from sklearn.metrics import accuracy_score

print('accuracy score = {}'.format(accuracy_score(y_test, y_pred)))

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix

cfsn_mat = confusion_matrix(y_test, y_pred)
print(cfsn_mat)

In [ ]:
plt.figure(figsize=(6,6))
plt.matshow(cfsn_mat, cmap='inferno');
# plt.axis('off')
plt.title('Confusion Matrix', y=1.1, size=16);
plt.colorbar();

In [ ]:
w_cnnclf = digits_cnnclf.get_weights()
digits_cnnclf.count_params()
len(w_cnnclf)
digits_cnnclf.summary()

In [ ]:

import matplotlib.pyplot as plt
%matplotlib inline
plt.matshow(w_cnnclf[0].reshape(12,12),cmap='inferno')

In [ ]:
digits_cnnclf.save('./simple-digits_cnnclf.h5')